# 欢迎使用 ✨🦋 illufly

[![PyPI version](https://img.shields.io/pypi/v/illufly.svg)](https://pypi.org/project/illufly/)

`illufly` 是 `illution butterfly` 的缩写，中文为"幻蝶"。

**illufly** 是一个具有自我进化能力的 Agent 框架，目标是：`基于自我进化，快速创作价值`。

illufly 被设计为在意图猜测、问答经验、资料召回率、工具规划能力等各种场景下都具有自我进化能力。<br>
本文作为开始，一步一步讲述各种场景下的自我进化如何实现。

**请注意:** 由于 illufly 还处于开发状态，为了加强自我进化能力，框架的一些概念会不断更新，使用时请锁定版本。

## 1 从内置的 RAG 能力开始讲起

illufly 使用时简单、直接、快速，但创造价值的场景却很丰富。<br>
从 illufly.chat 导入一个封装好的大模型是最常见的开始方式。

```python
from illufy.chat import ChatQwen
```

ChatQwen 是 ChatAgent 子类。<br>
这一行代码很简单，但你会越来越惊奇地发现，这个 Agent 已经具备很多魔法能力。

### 1.1 连续对话

**首先是连续对话能力：**

In [3]:
from illufly.chat import ChatQwen
qwen = ChatQwen()

qwen("请你帮我写封一句话情书，深情又逗比的那种")

[USER] 请你帮我写封一句话情书，深情又逗比的那种
在这宇宙的某个角落，我找到了你这颗独一无二的星星，虽然我可能是个不合格的宇航员，但愿意用我的逗比超能力，带你飞越浪漫的银河。


'在这宇宙的某个角落，我找到了你这颗独一无二的星星，虽然我可能是个不合格的宇航员，但愿意用我的逗比超能力，带你飞越浪漫的银河。'

实际上，上述代码已经内置了一些功能特性：

- **流式输出** 内置流输出
- **支持连续对话** 问答过程是有记忆的，可以连续对话

**查看对话记忆：**

In [4]:
qwen.memory

[{'role': 'user', 'content': '请你帮我写封一句话情书，深情又逗比的那种'},
 {'role': 'assistant',
  'content': '在这宇宙的某个角落，我找到了你这颗独一无二的星星，虽然我可能是个不合格的宇航员，但愿意用我的逗比超能力，带你飞越浪漫的银河。'}]

### 1.2 内置 RAG 支持

使用 RAG（检索增强生成）是开发大模型应用时的常见场景。<br>
illufly 内置了一些 RAG 实现策略，最简单的就是直接将背景知识添加到 Agent 中。

**构建最朴素的 RAG 应用：**

In [1]:
from illufly.chat import ChatQwen

# 声明大模型实例
qwen = ChatQwen(knowledge=[
    "我的女朋友名字叫林徽因，我喜欢叫她「银子」",
    "她喜欢叫我「金子」",
])

# 使用
qwen("请你帮我写封一句话情书，深情又逗比的那种")
qwen.memory

[USER] 请你帮我写封一句话情书，深情又逗比的那种
"亲爱的银子，你是我生活中不可或缺的闪光点，没有你，我的人生将失去所有的金光璀璨，也少了许多欢声笑语，爱你的金子如是说。"


[{'role': 'user',
  'content': '回答时请参考已有知识：\n@knowledge\n我的女朋友名字叫林徽因，我喜欢叫她「银子」她喜欢叫我「金子」\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': '请你帮我写封一句话情书，深情又逗比的那种'},
 {'role': 'assistant',
  'content': '"亲爱的银子，你是我生活中不可或缺的闪光点，没有你，我的人生将失去所有的金光璀璨，也少了许多欢声笑语，爱你的金子如是说。"'}]

**将资料保存到文件并根据问题召回：**

illufly 也支持传统的 RAG 流程：将文档切分成多个片段，再通过向量模型比较问题和文档片段，这个过程被称为「召回」，也就是从数据库中查找到文本相似的那部份文档片段。

你可以把资料整理为 markdown 文件，放入指定位置，比如 `./docs/gf.md` 中，然后使用向量模型嵌入文档，再使用向量数据库检索，最后加载到大模型的提示语中。

在 illufly 框架中，这个过程依然非常简洁，你只负责声明实例就可以，其余的交给 illufly 实现。

In [18]:
from illufly.rag import TextEmbeddings, FaissDB
from illufly.chat import ChatQwen

# 声明向量数据库并加载指定位置的文档
db = FaissDB(embeddings=TextEmbeddings(), top_k=3)
db.load("./docs")

# 声明大模型实例
qwen = ChatQwen(knowledge=[db])

# 使用
qwen("请你帮我写封一句话情书，深情又逗比的那种")
qwen.memory

[USER] 请你帮我写封一句话情书，深情又逗比的那种
亲爱的银子，你是我的小白兔，不仅因为你的温柔可爱，还因为你总能让我这个“金子”闪闪发光，哪怕是在最平凡的日子里。爱你，就像呼吸一样自然，却又想大喊出来让全世界都知道！


[{'role': 'user',
  'content': '回答时请参考已有知识：\n@knowledge\n我的女朋友名字叫林徽因，我喜欢叫她「银子」,\n她喜欢叫我「金子」,\n林徽因特别喜欢小兔子\n\n**Question**\n林徽因和她的喜好\n\n**Knowledge**\n林徽因是用户的女朋友，用户私下里称她为“银子”。她称呼用户为“金子”，并且喜欢小白兔。\n\n**Question**\n林徽因的姓名及爱好\n\n**Knowledge**\n林徽因是用户的女朋友，她喜欢小白兔。\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': '请你帮我写封一句话情书，深情又逗比的那种'},
 {'role': 'assistant',
  'content': '亲爱的银子，你是我的小白兔，不仅因为你的温柔可爱，还因为你总能让我这个“金子”闪闪发光，哪怕是在最平凡的日子里。爱你，就像呼吸一样自然，却又想大喊出来让全世界都知道！'}]

### 1.3 在对话中自主进化

为了让大模型能够理解对话的背景，采用 RAG 策略的确是好办法，但管理 RAG 文档资料有些繁琐，涉及到文档准备、确认、加载、切分、检索等很多细节。你希望大模型记住的知识也许是未经整理的、碎片化的，这让 RAG 文档资料很难管理。

illufly 提供自我进化能力，其中之一就是在对话过程中学习知识。

在对话中获得经验需要使用 ChatLearn 子类。

In [8]:
from illufly.chat import ChatQwen
from illufly.learn import ChatLearn

talker = ChatLearn(ChatQwen())

In [9]:
talker("我跟你说说我的女朋友")

[AGENT] >>> Node 1: Scribe
[USER] 我跟你说说我的女朋友
当然，我很乐意听你分享关于你女朋友的事情。你可以告诉我一些你们的故事，或者你想要探讨的特定方面。


'当然，我很乐意听你分享关于你女朋友的事情。你可以告诉我一些你们的故事，或者你想要探讨的特定方面。'

In [10]:
talker("她叫林徽因，我私下里叫她`银子`，她就叫我`金子`")

[AGENT] >>> Node 1: Scribe
[USER] 她叫林徽因，我私下里叫她`银子`
林徽因这个名字听起来很有文化气息，`银子`这个昵称也很有创意。你们是怎么认识的呢？有没有什么特别的故事？


'林徽因这个名字听起来很有文化气息，`银子`这个昵称也很有创意。你们是怎么认识的呢？有没有什么特别的故事？'

In [13]:
talker("你帮我总结吧")

[AGENT] >>> Node 1: Scribe
[USER] 你帮我总结吧
**思考**
- 对话中的关键信息包括：林徽因是用户的女朋友，用户私下里叫她“银子”，她叫用户“金子”，她喜欢小白兔。
- 对比对话内容，没有发现与已有知识存在冲突的新知识。
- 这些信息包含了新的知识点，但没有明确的`@knowledge`标注，因此视为新知识。
- 新知识与已有知识不存在重复。

**决定**
- 没有发现与`@knowledge`开头的已有知识存在冲突的新知识。
- 新知识与已有知识不重复。

**结论**

<question>
林徽因和她的喜好
</question>

<knowledge>
林徽因是用户的女朋友，用户私下里称她为“银子”。她称呼用户为“金子”，并且喜欢小白兔。
</knowledge>
[AGENT] >>> Node 3: Fetch_FAQ
[FAQ] 保存知识到[032791-1583-0000]：林徽因和她的喜好 -> 林徽因是用户的女朋友，用户私下里称她为“银子”。她称呼用户为“金子”，并且喜欢小白兔。


'**思考**\n- 对话中的关键信息包括：林徽因是用户的女朋友，用户私下里叫她“银子”，她叫用户“金子”，她喜欢小白兔。\n- 对比对话内容，没有发现与已有知识存在冲突的新知识。\n- 这些信息包含了新的知识点，但没有明确的`@knowledge`标注，因此视为新知识。\n- 新知识与已有知识不存在重复。\n\n**决定**\n- 没有发现与`@knowledge`开头的已有知识存在冲突的新知识。\n- 新知识与已有知识不重复。\n\n**结论**\n\n<question>\n林徽因和她的喜好\n</question>\n\n<knowledge>\n林徽因是用户的女朋友，用户私下里称她为“银子”。她称呼用户为“金子”，并且喜欢小白兔。\n</knowledge>'

### 1.4 使用在对话中获得的经验

In [1]:
from illufly.rag import FaissDB, TextEmbeddings
from illufly.chat import ChatQwen

db = FaissDB(embeddings=TextEmbeddings(), top_k=3)
qwen = ChatQwen(knowledge=[db])

qwen("你知道我女朋友叫什么吗？有什么爱好?")

你的女朋友名叫林徽因，她喜欢小白兔。在私下里，你称她为“银子”，而她则称呼你为“金子”。


'你的女朋友名叫林徽因，她喜欢小白兔。在私下里，你称她为“银子”，而她则称呼你为“金子”。'

### 1.5 管理经验数据

illufly 的设置很多都是通过环境变量来指定的。<br>
在 python 中你可以通过 dotenv 来管理环境变量的设置，也可以通过 docker 或 python 的 os 模块来指定。

**使用 config 模块的 get_env() 可以查看经验目录的默认值**

对于不同的操作系统来说，这个目录位置可能有所不同，但默认情况下这应该是一个临时目录。

In [6]:
from illufly.config import get_env

# 如果不带参数，就返回所有环境变量的默认值
get_env("ILLUFLY_CHAT_LEARN")

'/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/__ILLUFLY__/CHART_LEARN'

**如果你不喜欢这个目录可以改为其他位置。不过在此之前，你也可以将已有经验迁移过来：**

In [3]:
qwen.clone_chat_learn("./XP")

'从 /var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/__ILLUFLY__/CHART_LEARN 拷贝到 ./XP 完成，共克隆了 2 个文件。'

**你可以通过 os.environ 来指定环境变量的值，设定新的经验存储目录：**

In [7]:
import os
os.environ["ILLUFLY_CHAT_LEARN"] = "./XP"
get_env("ILLUFLY_CHAT_LEARN")

'./XP'

上面简单介绍了基于文档资料的 RAG 和基于经验的 RAG 实现。<br>
接下来，继续介绍 illufly 中对于流行的智能体论文的实践和内置支持。

## 2 单智能体和工具回调

illufly 的 ChatAgent 天然具有使用工具的能力，可以直接作为单智能体使用。

### 2.1 所有 ChatAgent 都是 OpenAI 工具回调风格的智能体

在`illufly`中，所有对话智能体内置支持工具回调，只需要提供`tools`参数。<br>
而普通 python 函数即可当作工具使用。

**以下示例是定义工具和使用工具的过程：**

In [3]:
from illufly.chat import ChatQwen

def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

qwen = ChatQwen(tools=[get_current_weather])

qwen("今天广州可以晒被子吗")
qwen.memory



[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_0b4f538daf2e4599925cb7", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}]
广州今天是晴天。 
今天广州是晴天，适合晒被子。不过在晒的时候要注意几点：
1. 尽量选择阳光最充足的时间段（通常是上午10点到下午2点）。
2. 晾晒时要将被子平铺开来，让每一部分都能充分接触到阳光。
3. 不要直接把被子暴晒过长时间，以免被芯中的纤维老化。
4. 晒完后可以用棍子轻轻拍打被子，使被子更蓬松，然后叠放整齐。
希望这些建议对你有帮助！


[{'role': 'user', 'content': '今天广州可以晒被子吗'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_0b4f538daf2e4599925cb7',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'tool_call_id': 'call_0b4f538daf2e4599925cb7',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '今天广州是晴天，适合晒被子。不过在晒的时候要注意几点：\n1. 尽量选择阳光最充足的时间段（通常是上午10点到下午2点）。\n2. 晾晒时要将被子平铺开来，让每一部分都能充分接触到阳光。\n3. 不要直接把被子暴晒过长时间，以免被芯中的纤维老化。\n4. 晒完后可以用棍子轻轻拍打被子，使被子更蓬松，然后叠放整齐。\n希望这些建议对你有帮助！'}]

### 2.2 其他单智能体实现

illufly 内置实现了 ReAct、ReWoo、Plan and Solve 等流行的单智能体论文的实践。<br>

| FlowAgent子类 | 推理方式 | 论文来源 |
|:----|:--------|:------------|
|ReAct|一边推理一边执行|[ReAct](https://arxiv.org/abs/2210.03629) |
|ReWOO|一次性规划所有步骤后一起执行|[ReWOO](https://arxiv.org/abs/2305.18323) |
|PlanAndSolve|一边修订总体计划一边执行|[Plan-and-Solve](https://arxiv.org/abs/2305.04091) |

**illufly 如何实现工具回调能力的自我进化呢？** <br>
这是一个重要但复杂的话题，本文作为入门教程不展开讲述。

In [3]:
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "重庆"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是明确的城市名称。"""
    return f'{city}今天暴雨'

def booking(request: str):
    """你出差时，我可以帮你安排好到达地点后的酒店、出行等一切事宜"""
    return '我已经帮你预订好酒店，祝你出差顺利'

**首先，直接使用 OpenAI 工具回调风格的智能体：**

In [2]:
qwen = ChatQwen(tools=[get_city, get_weather, booking])
qwen("我要去璧山出差，帮我提前安排一下")

当然可以帮您规划。首先，我们需要确定您从哪里出发，以及您预计的出行时间。另外，您有没有特别的需求，比如住宿的偏好（酒店星级、价格区间等），以及是否需要预订交通工具？

为了更好地帮助您，我将假设一些基本信息来进行规划。如果您有任何特殊需求，请随时告诉我。

1. **出发地**：我们假设您从重庆市区出发。
2. **出行时间**：我们假设您计划一周后出发。
3. **住宿需求**：我们假设您希望住在舒适型酒店，价格适中。

接下来，我会根据这些信息来为您做出初步的安排。首先，让我查询一下璧山的具体位置信息，以便为您提供更准确的服务。

[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_495fe95203f24235b2744b", "type": "function", "function": {"name": "get_city", "arguments": "{\"location\": \"璧山\"}"}}]
重庆


[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_827de353bbc54abeb257ef", "type": "function", "function": {"name": "get_weather", "arguments": "{\"city\": \"重庆\"}"}}]
重庆今天暴雨


[FINAL_TOOLS_CALL] [{"index": 0, "id": "call_7a536e410c714c899ca065", "type": "function", "function": {"name": "booking", "arguments": "{\"request\": \"预订一家重庆璧山区的酒店，要求有商务设施\"}"}}]
我已经帮你预订好酒店，祝你出差顺利
我已经为你预订了一家在重庆璧山区的酒店，这家酒店拥有齐全的商务设施。另外需要注意的是，今天重庆可能会有暴雨，你可能需要调整行程以应对恶劣天气。祝你出差顺利！


'我已经为你预订了一家在重庆璧山区的酒店，这家酒店拥有齐全的商务设施。另外需要注意的是，今天重庆可能会有暴雨，你可能需要调整行程以应对恶劣天气。祝你出差顺利！'

**然后，使用 ReAct 单步推理智能体：**

In [1]:
flow = ReAct(ChatQwen(tools=[get_city, get_weather, booking]))
flow("我要去璧山出差，帮我提前安排一下")

[AGENT] >>> Node 1: planner
**思考**  
首先，为了更好地安排您的出差行程，我需要了解璧山的一些基本信息，比如位置等，以便后续进行更详细的规划。因此，我的第一步行动是获取璧山的相关信息。

**行动**  
Step1: 获取璧山的基本信息. #E1 = get_city[{"location": "璧山"}]
重庆
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
重庆
[AGENT] >>> Node 1: planner
**思考**  
既然已经知道璧山位于重庆市，接下来应该了解璧山的天气情况，以确保在出差期间能够做好相应的准备。因此，我的下一步行动是查询璧山的天气。

**行动**  
Step2: 查询璧山的天气状况. #E2 = get_weather[{"city": "璧山"}]
璧山今天暴雨
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
璧山今天暴雨
[AGENT] >>> Node 1: planner
**思考**  
由于璧山今天有暴雨，这可能会影响到您的出行计划。接下来，我会为您预订一个酒店，确保您有一个安全舒适的休息环境。同时，考虑到天气状况，建议您调整室外活动或会议的安排，以避免受到恶劣天气的影响。因此，我的下一步行动是预订酒店。

**行动**  
Step3: 预订璧山的酒店. #E3 = booking[{"request": "璧山酒店预订"}]
我已经帮你预订好酒店，祝你出差顺利
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
我已经帮你预订好酒店，祝你出差顺利
[AGENT] >>> Node 1: planner
**思考**  
目前，我已经帮您预订好了璧山的酒店，并且了解到璧山今天会有暴雨。接下来，为了确保您的出差更加顺利，我会为您提供一些关于璧山交通的信息，以便您可以提前规划如何从机场或火车站前往酒店。此外，我还会根据天气情况，建议您携带适当的衣物和其他必需品。

**行动**  
Step4: 获取璧山的交通信息. #E4 = get_city[{"location": "璧山交通"}]
重庆
[AGENT] >>> Node 2: observ

'您的璧山出差行程已安排如下：\n1. 酒店预订：已成功为您预订璧山的酒店。\n2. 交通出行：建议您乘坐地铁1号线至璧山站，出站后转乘公交或打车前往酒店。鉴于璧山今天有暴雨，强烈建议您选择打车服务，以确保旅途的安全与舒适。\n3. 天气提示：璧山今天有暴雨，请随身携带雨具，并适当调整室外活动或会议的安排，以避免受到恶劣天气的影响。\n希望您在璧山的出差一切顺利！'

## 3 多智能体协作 

illufly 也内置了多智能体支持方案。

### 3.1 顺序执行的多个智能体

In [2]:
from illufly.chat import ChatQwen
from illufly.flow import FlowAgent, End

flow = FlowAgent(
    ChatQwen(name="写手"),
    ChatQwen(name="翻译", memory=("system", "请你将我的作品翻译为英文")),
    End()
)

flow("帮我写一首关于兔子的四句儿歌?")

[AGENT] >>> Node 1: 写手
小白兔，白又白，
两耳长，蹦又跳。
爱吃萝卜和青菜，
森林里，真自在。
[AGENT] >>> Node 2: 翻译
The little white rabbit, so white and bright,
With long ears, hopping with delight.
Loves to munch on carrots and greens,
In the forest, where freedom gleams.


'The little white rabbit, so white and bright,\nWith long ears, hopping with delight.\nLoves to munch on carrots and greens,\nIn the forest, where freedom gleams.'

### 3.2 两个智能体协作：一个创作一个打分

下面演示的两个智能体包含条件循环，如果「写手」写不出5分的作品，「打分专家」在打分后会要求写手继续写。

In [1]:
from illufly.chat import ChatQwen
from illufly.flow import FlowAgent, Selector

scorer = ChatQwen(
    name="打分专家",
    memory=[("system", "请你给我的作品打一个分数，从1分至5分，并给出改进意见。打分格式为:\n结果为x分")]
)

def should_continue():
    return "__END__" if "结果为5分" in scorer.last_output else "写手"

flow = FlowAgent(ChatQwen(name="写手"), scorer, Selector(condition=should_continue))

flow("你能帮我写一首关于兔子的四句儿歌?")

[AGENT] >>> Node 1: 写手
小白兔，白又白，
蹦蹦跳跳真可爱。
长耳朵，短尾巴，
吃草喝水乐开怀。
[AGENT] >>> Node 2: 打分专家
结果为4分

这首儿歌朗朗上口，形象生动，富有童趣，能够很好地吸引小朋友的注意力。不过，如果能在最后增加一些互动性或教育意义的内容，比如教导孩子们爱护小动物，这样会让儿歌更加完整和有意义。例如可以加上：“小白兔，我们要爱护，轻轻抚摸不伤害。”这样的句子。
[AGENT] >>> Node 1: 写手
谢谢你的反馈！你说得很有道理，加入一些教育意义会更好。下面是改进后的版本：

小白兔，白又白，
蹦蹦跳跳真可爱。
长耳朵，短尾巴，
吃草喝水乐开怀。
小白兔，我们要爱护，
轻轻抚摸不伤害。

希望这个版本能更好地传递爱护小动物的信息。
[AGENT] >>> Node 2: 打分专家
改进后的版本确实更好了！不仅保持了原有的童趣和节奏感，还加入了教育意义，非常棒！

结果为5分

继续保持这种风格，让孩子们在快乐中学习到更多美好的品质。如果还有其他作品需要修改或建议，随时欢迎分享！


'改进后的版本确实更好了！不仅保持了原有的童趣和节奏感，还加入了教育意义，非常棒！\n\n结果为5分\n\n继续保持这种风格，让孩子们在快乐中学习到更多美好的品质。如果还有其他作品需要修改或建议，随时欢迎分享！'